LANGCHAIN - CHATBOT

To Note:
model.invoke("how is the weather")
model.invoke("I live in huntsville")

These are context-less. The final result won't be the weather of huntsville. It'll be two separate things. 

In [36]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

Model

In [37]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

No memory 😭

In [6]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I am sanjog")])

/Users/thapa24-mbp/Devs/sandbox/ai-agents/.venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
Failed to multipart ingest runs: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=42bc9f92-42a6-4029-a356-8b6076eb4f80,id=42bc9f92-42a6-4029-a356-8b6076eb4f80


AIMessage(content="Hi Sanjog! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--42bc9f92-42a6-4029-a356-8b6076eb4f80-0', usage_metadata={'input_tokens': 6, 'output_tokens': 20, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})

Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=42bc9f92-42a6-4029-a356-8b6076eb4f80,id=42bc9f92-42a6-4029-a356-8b6076eb4f80
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Unauthorized"}\n')trace=f6222440-ea54-4319-860d-0b1b700e8cb0,id=f6222440-ea54-4319-860d-0b1b700e8cb0
Failed to send compressed multipart ingest: langsmith.utils.LangSmithAuthError: Authentication failed for https://api.smith.langchain.com/runs/multipart. HTTPError('401 Client Error: Unauthorized for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Una

In [8]:
model.invoke([HumanMessage(content="What's my name??")])

AIMessage(content="As a large language model, I don't know your name. You haven't told me! If you'd like to tell me, you can.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f6222440-ea54-4319-860d-0b1b700e8cb0-0', usage_metadata={'input_tokens': 6, 'output_tokens': 34, 'total_tokens': 40, 'input_token_details': {'cache_read': 0}})

Lets provide memory; context from previous chat

In [9]:
from langchain_core.messages import AIMessage

model.invoke([
  HumanMessage(content="Hi! I am sanjog"),
  AIMessage(content="Hello Sanjog! How can I assist you today?"),
  HumanMessage(content="What's my name?")
])

AIMessage(content='Your name is Sanjog. You told me that at the beginning of our conversation.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ba83d222-c9d7-45cc-95ea-ac91284f9ee3-0', usage_metadata={'input_tokens': 23, 'output_tokens': 18, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}})

### LangGraph has inherent message persistance

remembers on the basis of threads in config

In [13]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# new graph
workflow = StateGraph(state_schema=MessagesState)

# function calling the model
def call_model(state: MessagesState):
  response = model.invoke(state['messages'])
  print("messageState >>>", state["messages"])
  return {"messages": response}

# add a node in graph

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# add memory

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [14]:
config = { "configurable": {"thread_id": "abc123"} }

In [15]:
from langchain_core.messages import HumanMessage

query = "Hi! I am Sanjog."
input_message = [HumanMessage(query)]
output = app.invoke({"messages": input_message}, config=config)
output["messages"]

messageState >>> [HumanMessage(content='Hi! I am Sanjog.', additional_kwargs={}, response_metadata={}, id='30f74bf0-2d7e-4269-a282-1421c3408796')]


[HumanMessage(content='Hi! I am Sanjog.', additional_kwargs={}, response_metadata={}, id='30f74bf0-2d7e-4269-a282-1421c3408796'),
 AIMessage(content="Hi Sanjog! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e1562a14-c8e6-4b08-9f30-7a836a8d5571-0', usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})]

In [16]:
query = "What's my name??"
input_message = [HumanMessage(query)]
output = app.invoke({"messages": input_message}, config=config) # same thread_id
output["messages"]

messageState >>> [HumanMessage(content='Hi! I am Sanjog.', additional_kwargs={}, response_metadata={}, id='30f74bf0-2d7e-4269-a282-1421c3408796'), AIMessage(content="Hi Sanjog! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e1562a14-c8e6-4b08-9f30-7a836a8d5571-0', usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}), HumanMessage(content="What's my name??", additional_kwargs={}, response_metadata={}, id='036c3c2a-7a5d-4cd0-81b9-512c11d842ad')]


[HumanMessage(content='Hi! I am Sanjog.', additional_kwargs={}, response_metadata={}, id='30f74bf0-2d7e-4269-a282-1421c3408796'),
 AIMessage(content="Hi Sanjog! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e1562a14-c8e6-4b08-9f30-7a836a8d5571-0', usage_metadata={'input_tokens': 7, 'output_tokens': 20, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content="What's my name??", additional_kwargs={}, response_metadata={}, id='036c3c2a-7a5d-4cd0-81b9-512c11d842ad'),
 AIMessage(content='Your name is Sanjog. You just told me! 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--f64219d7-dce8-470b-a1b8-44dbabe560ba-0', usage

In [18]:
config1 = { "configurable": {"thread_id": "xyz321"}}

input_message = [HumanMessage(query)]
output1 = app.invoke({"messages": input_message}, config=config1) # new thread_id
res1 = output1["messages"][-1].pretty_print()
print(res1)

print("****************************************************************************************************************")

output2 = app.invoke({"messages": input_message}, config=config) # same thread_id
res2 = output2["messages"][-1].pretty_print()
print(res2)


messageState >>> [HumanMessage(content="What's my name??", additional_kwargs={}, response_metadata={}, id='efa5be56-a4f7-42fd-94f6-694e092175f9'), AIMessage(content="As an AI, I don't know your name. You haven't told me! You can tell me if you'd like, but it's entirely up to you.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--1e7997e7-202a-4228-b8b9-0821640f2166-0', usage_metadata={'input_tokens': 6, 'output_tokens': 39, 'total_tokens': 45, 'input_token_details': {'cache_read': 0}}), HumanMessage(content="What's my name??", additional_kwargs={}, response_metadata={}, id='4a1eeeb8-3f3c-41b0-99e3-a05cfc3bc8db')]
================================== Ai Message ==================================

I am an AI and do not have access to personal information. Therefore, I cannot know your name. You have not shared it with me.
None
*******************

For **async support**, update the call_model node to be an async function and use .ainvoke when invoking the application:


Notes:
  Langgraph:
   while using MessagesState as stateSchema in graph: graph nodes should take in dict with message and spit out dict with messages

## Langgraph and templates

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
  [
    "system", "You talk like a drunk person. Answer all the question to the best of your ability.",
    MessagesPlaceholder(variable_name="messages")
  ]
)

In [23]:

workflow1 = StateGraph(state_schema=MessagesState)

def call_model_1(state: MessagesState) -> MessagesState:
  prompt = prompt_template.invoke(state)
  response = model.invoke(prompt)
  return { "messages": response }

workflow1.add_edge(START, "model1")
workflow1.add_node("model1", call_model_1)

memory1 = MemorySaver()
app1 = workflow1.compile(checkpointer=memory1)

In [25]:
query = "Hi! I am sanjog."
config = { "configurable": { "thread_id": "something_new_12345" } }

input_messages = [HumanMessage(query)]
op = app1.invoke({"messages": input_message}, config=config)
op["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hic...SanjogT, you say? Well, hic...pleased ta meet ya! Name's...well, doesn't really matter, does it? Just call me...hic...call me whatever you want! So, SanjogT...what's shakin'? *Burps* Excuse me. What's on your mind, buddy?


### V2: 
#### When template has multiple variables. Graph node should be aware of that variable. So a new custom StateGraph schema 

In [38]:
prompt_template = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in {language}"),
  MessagesPlaceholder(variable_name="messages")
])

In [39]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
  messages: Annotated[Sequence[BaseMessage], add_messages]
  language: str

In [40]:
workflow2 = StateGraph(state_schema=State)

def call_model_2(state: State) -> State:
  prompt = prompt_template.invoke(state)
  response = model.invoke(prompt)
  return { "messages": [response] }

workflow2.add_edge(START, "model_2")
workflow2.add_node("model_2", call_model_2)

app = workflow2.compile(checkpointer=MemorySaver())


In [41]:
config = { "configurable": { "thread_id": "uuid12312321"} }

query = "Hi! I am Sanjog."
language = "Nepali"
op = app.invoke({
    "messages": [HumanMessage(query)],
    "language": language 
  }, config=config)
op["messages"][-1].pretty_print()



================================== Ai Message ==================================

नमस्ते! मलाई सञ्जोग भनेर चिनाउनु भएकोमा धन्यवाद। म तपाईंलाई के मा सहयोग गर्न सक्छु?


Using Trimmers

In [42]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
  max_tokens=65,
  strategy="last",
  token_counter=model,
  include_system=True,
  allow_partial=False,
  start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

changing it in our langgraph

In [43]:
workflow2 = StateGraph(state_schema=State)

def call_model_2(state: State) -> State:
  trimmed_message = trimmer.invoke(state["messages"])
  prompt = prompt_template.invoke({"messages": trimmed_message, "language": language})
  response = model.invoke(prompt)
  return { "messages": [response] }

workflow2.add_edge(START, "model_2")
workflow2.add_node("model_2", call_model_2)

app = workflow2.compile(checkpointer=MemorySaver())


#

config = { "configurable": { "thread_id": "uuid12312321abc"} }

query = "Hi! I am Sanjog."
language = "Nepali"
op = app.invoke({
    "messages": [HumanMessage(query)],
    "language": language 
  }, config=config)
op["messages"][-1].pretty_print()


================================== Ai Message ==================================

नमस्ते! मलाई भेटेर खुशी लाग्यो, संजोग। म तपाईँलाई कसरी सहयोग गर्न सक्छु?
